Notebook for generating train and test data from WT2 and WT103 datasets.

Sources:

https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0

https://towardsdatascience.com/running-jupyter-notebook-on-the-cloud-in-15-mins-azure-79b7797e4ef6

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import WikiText2
import pandas as pd
from nltk.corpus import brown
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import sys
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import os
import random

In [67]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-2-v1")


In [139]:
DATA_SPLIT = "test"
text = wikitext2[DATA_SPLIT]['text']
text = [item.lower().strip() for item in text if len(item) > 0]
len(text)

2891

In [140]:
text = [item.split(" ") + ["\n"] for item in text if "=" not in item]

In [130]:
# Prepare Corpus
DATA_LIMIT = None #paragraph limit
all_words = []
for paragraph in text[:DATA_LIMIT]:
    all_words += paragraph
all_words = pd.Series(all_words)
len(all_words)

2023109

In [131]:
len(all_words.unique())


28909

In [132]:
unique_paragraphs = [pd.Series(p).unique() for p in text]
l = []
for p in unique_paragraphs:
    l += list(p)
v_counts = pd.Series(l).value_counts()

In [133]:
#filter out rare words
N_times = 100 #shows up in 100 different paragraphs
# v_counts = all_words.value_counts()
corpus = pd.Series([key for key in v_counts.keys() if v_counts[key] >= N_times])
corpus

0                   \n
1                    ,
2                    .
3                  the
4                   of
             ...      
1751         sustained
1752          historic
1753           failure
1754    administration
1755              1987
Length: 1756, dtype: object

In [134]:
vocab = {}
for i in range(len(corpus)):
    vocab[corpus[i]] = i
len(vocab)

1756

In [135]:
version = "may1_WT2_nodatalim_20epoch_64dim_100minpf_2window"
torch.save(vocab, f"saves/vocab_{version}.pt")

In [141]:
enc_text = [[vocab[word] if word in vocab else vocab["<unk>"] for word in paragraph] for paragraph in text]
len(enc_text)

2176

In [142]:
SCANNING_WINDOW = 2
#maybe need to split into paragraphs b/c different topics...
#returns context, middle word
def get_data(index, window, data):
    return list(data[index-window:index])+list(data[index+1:index+window+1]), data[index]

In [143]:
x = []
y = []
count = 0
for paragraph in enc_text:
    if count % (len(enc_text)//16) == 0:
        print("•", end="")
    for i in range(SCANNING_WINDOW, len(paragraph) - SCANNING_WINDOW):
        if paragraph[i] == vocab["<unk>"]:
            continue
        a, b = get_data(i, SCANNING_WINDOW, paragraph)
        x.append(a)
        y.append(b)
    count+=1
    
torch.save(x, f"train_data/{DATA_SPLIT}_data_x_wt2_window2_100minpf.pt")
torch.save(y, f"train_data/{DATA_SPLIT}_data_y_wt2_window2_100minpf.pt")

••••••••••••••••